In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV, train_test_split

from sktime.classification.deep_learning.cnn import CNNClassifier

sns.set_style("whitegrid")

2022-08-10 16:40:13.298347: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-10 16:40:13.298401: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
AU_COLS = ['AU01_r',
'AU02_r',
'AU04_r',
'AU05_r',
'AU06_r',
'AU07_r',
'AU09_r',
'AU10_r',
'AU12_r',
'AU14_r',
'AU15_r',
'AU17_r',
'AU20_r',
'AU23_r',
'AU25_r',
'AU26_r',
'AU45_r']

ID_COLS = ['filename', "frame", "video_id", "emotion_1_id"]

In [3]:
load_dotenv()

# path to save figures
output_path = os.getenv("VIDEO_OUT")

load_path = os.path.join(output_path, 'video_data_time_series.csv')
df = pd.read_csv(load_path)

In [4]:
df

,filename,frame,video_id,emotion_1_id,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r
0,A050121-R_awe_p_3,1,A050121-R,18,0.000,0.000,0.146,0.000000,0.345455,0.174,...,0.250,0.322,0.010,0.000,0.372,0.008,0.046,0.000,0.306,0.058
1,A050121-R_awe_p_3,2,A050121-R,18,0.000,0.000,0.132,0.000000,0.343434,0.192,...,0.254,0.308,0.014,0.000,0.336,0.006,0.046,0.000,0.282,0.026
2,A050121-R_awe_p_3,3,A050121-R,18,0.000,0.000,0.126,0.000000,0.351515,0.206,...,0.264,0.304,0.016,0.000,0.314,0.006,0.040,0.000,0.264,0.008
3,A050121-R_awe_p_3,4,A050121-R,18,0.000,0.000,0.126,0.000000,0.361616,0.222,...,0.266,0.302,0.020,0.000,0.314,0.002,0.034,0.000,0.266,0.046
4,A050121-R_awe_p_3,5,A050121-R,18,0.000,0.000,0.138,0.000000,0.381818,0.274,...,0.266,0.306,0.020,0.008,0.318,0.002,0.022,0.000,0.282,0.134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2238014,A91_sad_p_3,196,A91,6,0.008,0.000,0.000,0.004435,0.000000,0.000,...,0.128,0.142,0.000,0.000,0.000,0.000,0.000,0.328,0.000,0.006
2238015,A91_sad_p_3,197,A91,6,0.000,0.000,0.000,0.000000,0.000000,0.000,...,0.114,0.134,0.000,0.008,0.004,0.000,0.000,0.308,0.000,0.008
2238016,A91_sad_p_3,198,A91,6,0.000,0.002,0.000,0.000000,0.000000,0.000,...,0.100,0.128,0.000,0.020,0.004,0.000,0.000,0.288,0.000,0.004
2238017,A91_sad_p_3,199,A91,6,0.000,0.020,0.000,0.000000,0.000000,0.000,...,0.100,0.128,0.000,0.034,0.010,0.000,0.000,0.276,0.000,0.004


In [5]:
X = []
y = []
for _, group in df.groupby('filename'):
    x_arr = np.asarray(group[AU_COLS])

    X.append(x_arr)
    
    y_list = group["emotion_1_id"].unique()
    if len(y_list) == 1:
        y.append(y_list[0])
    else:
        raise ValueError('More than 1 label for time series encountered')

In [6]:
length = max(map(len, X))

padded_X = []
for xi in X:
    pad = np.zeros((length-len(xi), xi.shape[1]))
    xi = np.concatenate((xi, pad)).T
    padded_X.append(xi)

In [7]:
y = np.asarray(y)
X = np.array(padded_X)

In [8]:
y

array([ 5,  5,  5, ..., 20, 20, 20])

In [31]:
X.shape

(10936, 17, 1549)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [10]:
network = CNNClassifier(n_epochs=200, verbose=True)
network.fit(X_train, y_train)
network.score(X_test, y_test)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1549, 17)]        0         
                                                                 
 conv1d (Conv1D)             (None, 1543, 6)           720       
                                                                 
 average_pooling1d (AverageP  (None, 514, 6)           0         
 ooling1D)                                                       
                                                                 
 conv1d_1 (Conv1D)           (None, 508, 12)           516       
                                                                 
 average_pooling1d_1 (Averag  (None, 169, 12)          0         
 ePooling1D)                                                     
                                                                 
 flatten (Flatten)           (None, 2028)              0     

2022-08-10 16:40:56.529405: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-10 16:40:56.529430: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-10 16:40:56.529451: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tim-Lenovo-Yoga-S730-13IWL): /proc/driver/nvidia/version does not exist
2022-08-10 16:40:56.530044: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/200


2022-08-10 16:40:58.111290: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 806316460 exceeds 10% of free system memory.


479/479 [==============================] - 6s 11ms/step - loss: 0.0234 - accuracy: 0.0247
Epoch 2/200
479/479 [==============================] - 6s 12ms/step - loss: 0.0225 - accuracy: 0.0225
Epoch 3/200
479/479 [==============================] - 6s 12ms/step - loss: 0.0223 - accuracy: 0.0227
Epoch 4/200
479/479 [==============================] - 6s 12ms/step - loss: 0.0223 - accuracy: 0.0230
Epoch 5/200
479/479 [==============================] - 6s 12ms/step - loss: 0.0222 - accuracy: 0.0266
Epoch 6/200
479/479 [==============================] - 6s 13ms/step - loss: 0.0222 - accuracy: 0.0357
Epoch 7/200
479/479 [==============================] - 6s 12ms/step - loss: 0.0221 - accuracy: 0.0453
Epoch 8/200
479/479 [==============================] - 6s 13ms/step - loss: 0.0220 - accuracy: 0.0477
Epoch 9/200
479/479 [==============================] - 6s 13ms/step - loss: 0.0220 - accuracy: 0.0498
Epoch 10/200
479/479 [==============================] - 6s 13ms/step - loss: 0.0220 - accuracy

2022-08-10 17:03:56.320554: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 345594292 exceeds 10% of free system memory.


206/206 [==============================] - 1s 5ms/step


0.06339530630905212